In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [8]:
# Load the csv exported in Part I to a DataFrame

weather_data = pd.read_csv('weather_data.csv')
weather_data.head()

,Unnamed: 0,City,Lat,lon,Temp_max,Humidity,Cloudiness,Wind_speed,Country,Date
0,0,Dikson,73.5069,80.5464,-0.98,58,100,10.90,RU,1633825055
1,1,Kavieng,-2.5744,150.7967,28.63,71,83,3.95,PG,1633824758
2,2,Ribeira Grande,38.5167,-28.7000,21.84,81,76,9.58,PT,1633825058
3,3,Avenal,36.0041,-120.1290,24.50,27,0,4.85,US,1633825059
4,4,Namatanai,-3.6667,152.4333,29.02,69,66,3.14,PG,1633825060


In [36]:
weather_lat_lon = weather_data[["Lat", "lon"]]
# weather_lat_lon
# records = weather_lat_lon.to_records(index=False)

# coordinates = list(records)
# type(coordinates)

list

In [168]:
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations =weather_data[["Lat", "lon"]].astype(float)

humidity = weather_data["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig
    

Figure(layout=FigureLayout(height='420px'))

In [144]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values

weather_data= weather_data.dropna()
# Temperature between 17 C and 22 C
# Wind speed less than 10 kph
# cloudiness 0 %
# humidity less than 45%


x =weather_data.loc[(weather_data["Temp_max"] > 17) & (weather_data["Temp_max"] < 22) & (weather_data["Cloudiness"] < 10) &(weather_data["Wind_speed"] < 10) &(weather_data["Humidity"] < 35)]
x= pd.DataFrame(x)
x

,Unnamed: 0,City,Lat,lon,Temp_max,Humidity,Cloudiness,Wind_speed,Country,Date
54,54,Erzin,36.9559,36.2000,20.56,31,0,1.34,TR,1633825126
283,283,Fort Morgan,40.2503,-103.8000,20.36,17,1,7.72,US,1633825448
441,441,Ghanzi,-21.5667,21.7833,18.39,12,0,5.10,BW,1633825670
481,481,Raton,36.9034,-104.4391,20.19,28,1,6.17,US,1633825663


In [183]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = x
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel lon"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
     

#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

# params dictionary to update each iteration
    my_params = {
        "location" : str(row["Lat"]) + ',' + str(row["lon"]),
        "radius": 5000,
        "keyword" : "hotel",
        "key": g_key
    }





    # make request 
    try:
        hotels = requests.get(base_url, params=my_params)
        #print(hotels)
        hotels = hotels.json()
        #print(hotels["results"][0]["name"])
        # choose the first hotel in the list
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"]=hotels["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel lon"]=hotels["results"][0]["geometry"]["location"]["lng"]
    except:
        hotel_df.loc[index, "Hotel Name"]="no hotel found with this search criteria"
        hotel_df.loc[index, "Hotel Lat"]=None
        hotel_df.loc[index, "Hotel lon"]=None
#print(json.dumps(hotels, indent=4, sort_keys=True))

    


In [182]:
hotel_df

,Unnamed: 0,City,Lat,lon,Temp_max,Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name,Hotel Lat,Hotel lon
54,54,Erzin,36.9559,36.2000,20.56,31,0,1.34,TR,1633825126,Hattusa Vacation Thermal Club Erzin,36.951793,36.250442
283,283,Fort Morgan,40.2503,-103.8000,20.36,17,1,7.72,US,1633825448,Hampton Inn Fort Morgan,40.260804,-103.800502
441,441,Ghanzi,-21.5667,21.7833,18.39,12,0,5.10,BW,1633825670,no hotel found with this search criteria,,
481,481,Raton,36.9034,-104.4391,20.19,28,1,6.17,US,1633825663,"Holiday Inn Express & Suites Raton, an IHG Hotel",36.865203,-104.441007


In [185]:
#drop any rows with null value
hotel_df= hotel_df.dropna()
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


gmaps.configure(api_key=g_key)

# Store 'Lat' and 'lon' into  locations 
location =hotel_df[["Hotel Lat", "Hotel lon"]].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(location, info_box_content=[hotel for hotel in hotel_df["Hotel Name"]])
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…